In [ ]:
#Ryan Kok
#This program is an administration system that handles creating, updating and deleting patient records


import sqlite3
from sqlite3 import Error
import ctypes
from IPython.display import clear_output

file_path = r"DonorDatabase.db"

#Create Connection function START
#Each large group of code has comments around it to signal its start and end
#Reusable function that is called whenever we need to interact with the database
def createConnection(dbfile):
    connect = None
    try:
        connect = sqlite3.connect(dbfile)
    except Error:
        print(Error)
    
    return connect 
#Create Connection function END

#Create Database Function START
#This function is used to create the donors table. Ideally should only need to be used once
def databaseCreate(file):
    
    connect = createConnection(file)
        
    sql_create_table = """CREATE TABLE IF NOT EXISTS donors
                    (
                    DonorID integer PRIMARY KEY,
                    Name text NOT NULL,
                    Surname text NOT NULL,
                    BloodType text NOT NULL,
                    Contact text NOT NULL
                    );"""
    
    try:
        cur = connect.cursor()
        cur.execute(sql_create_table)
        connect.close
    except:
        print("Table Creation Error")
        
        #Code that creates a windows pop up message to dispaly an error (Tak, 2022)
        ctypes.windll.user32.MessageBoxW(0, "Table Creation Error", "Error", 0) 
#Create Database Function END            

#Donor Register function START
#Function to add a donor record to the database
def donor_register():
    #Clears the terminal output (ipython, 2017)
    clear_output(wait=False)
    print("1. Donor Registration")
    print("---------------------------")
    
    connect = createConnection(file_path)
    cur = connect.cursor()
    #Selects the last row in the table, so that we can use the most recent DonorID to make the next DonorID
    # (TutorialsPoint, 2018)
    cur.execute("SELECT DonorID FROM donors ORDER BY DonorID DESC LIMIT 1")
    
    try:
        rawID = cur.fetchone()
        ID = rawID[0]
        ID = ID + 1
    except:
        ID = 1


    #Ask the admin to input each of the values of the new patient
    name = input("Please input the donor's first name: ")
    surname = input("Please input the donor's surname: ")
    bloodType = input("Please input the donor's blood type: ")
    contact = input("Please input the donor's contact number: ")
    values = (ID, name, surname, bloodType, contact)
    
    sql = """INSERT INTO donors(DonorID, Name, Surname, BloodType, Contact)
            VALUES(?, ?, ?, ?, ?)
            """
    
    
    cur.execute(sql, values)
    connect.commit()
    connect.close()
    
    ctypes.windll.user32.MessageBoxW(0, "Added 1 Patient Record", "Success", 0)
    
    
    flag1 = False
            
    #While Loop 1 START
    #Loop to check if another patient has to be added
    while flag1 == False:
        choice = input("\nAdd another patient?(Y/N): ")

        if choice == "y" or choice == "Y":            
            flag1 = True
            #If another patient needs to be added, use recursion to call the donor_register function again
            donor_register()                    
            return
        elif choice == "n" or choice == "N":
            flag1 = True                    
            mainMenu()
            return
        else:
            ctypes.windll.user32.MessageBoxW(0, "Invalid Input", "Invalid", 0)
            print("That input was invalid.")
            continue
     #While Loop 1 END

#Donor Register function END

#Donor Update function START
#Function that will search for a donor ID and then update the values for that donors record
def donor_update():
    clear_output(wait=True)
    print("2. Donor Update")
    print("---------------------------")
    
    connect = createConnection(file_path)
    cur = connect.cursor()
    
    flag1 = False
    
    #While Loop 1 START
    #First loop checks the database for the ID that was entered. The function will continue if the ID is found
    while flag1 == False:
        print("Please enter the ID of the patient you would like to update (type cancel to stop)")
        searchID = input("ID Number:")
        
        if searchID == "cancel" or searchID == "Cancel":
            print("\nCanceling updates.\n")
            mainMenu()
            return           
        
        sql = """SELECT * FROM donors WHERE DonorID = ?;"""
        
        cur.execute(sql, searchID)
        
        try:
            row = cur.fetchone()
            
            print("\nID\tName\tSurname\t\tBlood Type\tContact")
            print(str(row[0]) + "\t" + row[1] + "\t" + row[2] + "\t\t" + row[3] + "\t\t" + row[4] + "\n") #(Lemonaki, 2021)
            flag1 = True
            ctypes.windll.user32.MessageBoxW(0, "Patient was found", "Success", 0)
        except:
            ctypes.windll.user32.MessageBoxW(0, "Invalid Input", "Invalid", 0)
            print("That ID could not be found\n")
     #While Loop 1 END
    
    print("Which value would you like to update?")
    print("1. Name")
    print("2. Surname")
    print("3. Blood Type")
    print("4. Contact Number\n")
    
    flag2 = False
    
    #While Loop 2 START
    #Second loop is to see which value in the patient record has to be updated
    while flag2 == False:
        try:
            field = int(input("Option:"))
        except:
            print("That option is invalid")
            continue
        
        newValue = input("\nUpdated Value: ")
        
        if field == 1:
                sql = """UPDATE donors SET Name = ? WHERE DonorID = ?"""
                values = (newValue, searchID)
                
                cur.execute(sql, values)
                connect.commit()
                
                flag2 = True
        elif field == 2:
                sql = """UPDATE donors SET Surname = ? WHERE DonorID = ?"""
                values = (newValue, searchID)
                
                cur.execute(sql, values)
                connect.commit()
                
                flag2 = True
        elif field == 3:
                sql = """UPDATE donors SET BloodType = ? WHERE DonorID = ?"""
                values = (newValue, searchID)
                
                cur.execute(sql, values)
                connect.commit()
                
                flag2 = True
        elif field == 4:
                sql = """UPDATE donors SET Contact = ? WHERE DonorID = ?"""
                values = (newValue, searchID)

                cur.execute(sql, values)
                connect.commit()        

                flag2 = True
        else:
            ctypes.windll.user32.MessageBoxW(0, "Invalid Input", "Invalid", 0)
            print("That option is invalid")            
    #While Loop 2 END
    
    sql = """SELECT * FROM donors WHERE DonorID = ?;"""

    cur.execute(sql, searchID)


    row = cur.fetchone()

    print("\nUpdated Values:\n")
    print("ID\tName\tSurname\t\tBlood Type\tContact")
    print(str(row[0]) + "\t" + row[1] + "\t" + row[2] + "\t\t" + row[3] + "\t\t" + row[4] + "\n")
    ctypes.windll.user32.MessageBoxW(0, "Patient has been updated", "Success", 0)
    connect.close()
#Donor Update function END

#Donor Remove function START
#Function that will search for a donor ID and then delete that donor
def donor_remove():
    clear_output(wait=True)
    print("3. Donor Removal")
    print("---------------------------")
    
    connect = createConnection(file_path)
    cur = connect.cursor()
    
    flag1 = False
    
    #While Loop 1 START
    while flag1 == False:
        print("Please enter the ID of the patient you would like to remove (type cancel to stop)")
        searchID = input("ID Number:")
        
        if searchID == "cancel" or searchID == "Cancel":
            print("\nCanceling removal.\n")
            mainMenu()
            return           
        
        sql = """SELECT * FROM donors WHERE DonorID = ?;"""
        
        cur.execute(sql, searchID)
        
        try:
            row = cur.fetchone()
            
            print("\nID\tName\tSurname\t\tBlood Type\tContact")
            print(str(row[0]) + "\t" + row[1] + "\t" + row[2] + "\t\t" + row[3] + "\t\t" + row[4] + "\n")
                    
            
            flag2 = False
            
            #While Loop 2 START
            while flag2 == False:
                print("Are you sure you want to remove this patient?")
                choice = input("Y/N:")
                
                if choice == "y" or choice == "Y":
                    sql = """DELETE FROM donors WHERE DonorID = ?"""
                    
                    cur.execute(sql, searchID)
                    connect.commit()
                    print("Patient removed successfully\n")
                    connect.close()
                    flag2 = True
                    mainMenu()                    
                    return
                elif choice == "n" or choice == "N":
                    print("Canceling patient removal.\n")
                    connect.close()
                    flag2 = True                    
                    mainMenu()
                    return
                else:
                    ctypes.windll.user32.MessageBoxW(0, "Invalid Input", "Invalid", 0)
                    print("That input was invalid.")
                    continue
             #While Loop 2 END
            
        except:
            print("That ID could not be found\n")
     #While Loop 1 END
    
#Donor Remove function END

#Display Records function START
#Function for displaying all of the patient records
def displayRecords():
    clear_output(wait=True)
    print("4. Displaying Records")
    print("---------------------------")
    
    connect = createConnection(file_path)
    
    print("ID\tName\tSurname\t\tBlood Type\tContact")
    
    cur = connect.cursor()
    cur.execute("SELECT * FROM donors")
    rows = cur.fetchall() #(Vishal, 2021)
    
    for i in rows:
        print(str(i[0]) + "\t" + i[1] + "\t" + i[2] + "\t\t" + i[3] + "\t\t" + i[4])
        
    connect.close()
    
    enter = input("Press enter to continue\n")
    mainMenu()
    return
#Display Records function END
    
# Main Menu function START
#Displays the main menu
def mainMenu():
    clear_output(wait=True)
    print("Main Menu")
    
    print("---------------------------")
    print("1. Register Donor")
    print("2. Update Existing Record")
    print("3. Delete Record")
    print("4. Display Records")
    print("5. Exit Program")
    print("---------------------------")
    
    print("Please select an option from 1-5: ")
    choice = input("Option: ")
    print("")
    
    if int(choice) > 0 and int(choice) < 6:
        run(int(choice))
#Main Menu function END

#Run function START
#Runs all the functions for the main menu
def run(input):
    if input == 1:
        donor_register()
        
    elif input == 2:
        donor_update()
        
    elif input == 3:
        donor_remove()
        
    elif input == 4:
        displayRecords()
    
    elif input == 5:
        ctypes.windll.user32.MessageBoxW(0, "Exiting Program", "", 0)
        print("\nExiting Program")
        exit()
#Run function END    

if __name__ == "__main__":
    databaseCreate(file_path)
    mainMenu()

4. Displaying Records
---------------------------
ID	Name	Surname		Blood Type	Contact
1	Dylan	Kok		A+		064 232 1456
2	Byron	Kenzie		B+		064 192 1342
